In [1]:
!pip install transformers datasets torch nltk emoji


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [3]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("/content/drive/MyDrive/Copie de training.1600000.processed.noemoticon.csv", encoding='latin-1', header=None)
df.columns = ["target", "id", "date", "flag", "user", "text"]

# Convert sentiment labels 0,2,4 → 0,1,2
label_map = {0: 0, 2: 1, 4: 2}
df["label"] = df["target"].map(label_map)

df = df[["text", "label"]]  # keep only what's needed
df.head()


,text,label
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [4]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

# Train/validation split (90/10)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]


# 📊 BERTweet Sentiment140 Fine‑Tuning Summary

This notebook demonstrates how to fine‑tune the **BERTweet-base** model
on the **Sentiment140** dataset (1.6M tweets) using **PyTorch**,
**HuggingFace Transformers**, and **Weights & Biases (W&B)** for
experiment tracking and reporting.

------------------------------------------------------------------------

## 🚀 Project Overview

### **🔧 Model Used**

-   **BERTweet-base** (`vinai/bertweet-base`)
-   Pretrained transformer specifically optimized for **Twitter-style
    text**
-   Fine‑tuned for **3-way sentiment classification**:
    -   😡 Negative → 0\
    -   😐 Neutral → 1\
    -   😊 Positive → 2

------------------------------------------------------------------------

## ⚙️ Key Training Settings

  Component               Value
  ----------------------- ----------------------------------------
  Model                   `vinai/bertweet-base`
  Optimizer               AdamW
  Learning Rate           `2e-5`
  Batch Size              `64`
  Gradient Accumulation   `4 steps` (effective batch size = 256)
  Max Token Length        `64`
  Epochs                  `2`
  Scheduler               Linear Warmup (10%)
  Mixed Precision         Yes (AMP)
  Padding                 Dynamic (efficient memory use)

------------------------------------------------------------------------

## 📦 Dataset Preparation

-   Loaded the **Sentiment140 CSV** (1.6M rows)
-   Cleaned & mapped sentiment labels:\
    `0 → 0`, `2 → 1`, `4 → 2`
-   Only kept: **text + label**
-   Split: **90% train / 10% validation**
-   Applied:
    -   ⚡ Fast tokenization\
    -   🧹 Removal of unused `token_type_ids`
    -   🔢 PyTorch formatting for DataLoader

------------------------------------------------------------------------

## 📈 Experiment Tracking (W&B)

The training logs include: - Training loss\
- Learning rate curve\
- Validation accuracy\
- Final saved model as a **W&B Artifact**

Links allow: - Comparing runs\
- Viewing metrics\
- Exporting reports

------------------------------------------------------------------------

## 🧠 Training Pipeline Summary

1.  Load model + tokenizer\
2.  Tokenize dataset\
3.  Create DataLoaders\
4.  Initialize W&B\
5.  Run training loop with:
    -   AMP\
    -   Accumulated gradients\
    -   Scheduler\
    -   Frequent logging\
6.  Validate on test set\
7.  Save model + push to W&B

------------------------------------------------------------------------

## 💾 Output

After training completes: - Saved model directory:
**`bertweet_finetuned/`** - Saved tokenizer - W&B artifact for
reproducibility

------------------------------------------------------------------------

## ⭐ Final Notes

This workflow focuses on: - **Speed 🏎️** - **GPU efficiency ⚡** -
**Reproducibility 🔁** - **Stable training on a large dataset (1.6M
tweets)**

Perfect for sentiment research, benchmarking, and downstream deployment.

In [5]:
# ===========================
# 1. Install & Import
# ===========================
!pip install wandb datasets transformers accelerate

import wandb
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from datasets import Dataset
from tqdm import tqdm

# ===========================
# 2. W&B Login
# ===========================
wandb.login()


# ===========================
# 4. Load BERTweet Tokenizer & Model
# ===========================
checkpoint = "vinai/bertweet-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=3
)


# ===========================
# 5. Tokenization Function
# ===========================
def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding=False,         # FASTEST + LESS RAM
        max_length=64
    )

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset   = val_dataset.map(tokenize_fn, batched=True)


# ===========================
# 6. Remove token_type_ids (BERTweet does NOT use it)
# ===========================
def remove_token_type_ids(batch):
    batch.pop("token_type_ids", None)
    return batch

train_dataset = train_dataset.map(remove_token_type_ids)
val_dataset   = val_dataset.map(remove_token_type_ids)


# ===========================
# 7. Final Dataset Format
# ===========================
train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)


# ===========================
# 8. DataLoader
# ===========================
def collate_fn(batch):
    return {
        "input_ids": torch.nn.utils.rnn.pad_sequence([b["input_ids"] for b in batch], batch_first=True),
        "attention_mask": torch.nn.utils.rnn.pad_sequence([b["attention_mask"] for b in batch], batch_first=True),
        "labels": torch.tensor([b["label"] for b in batch], dtype=torch.long)
    }

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


# ===========================
# 9. W&B Experiment Setup
# ===========================
wandb.init(
    project="bertweet-sentiment140",
    config={
        "model": checkpoint,
        "epochs": 2,
        "batch_size": batch_size,
        "learning_rate": 2e-5,
        "accum_steps": 4,
        "max_length": 64
    }
)

config = wandb.config


# ===========================
# 10. Training Preparation
# ===========================
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

optimizer = AdamW(model.parameters(), lr=config.learning_rate)

total_steps = len(train_loader) // config.accum_steps * config.epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

scaler = torch.cuda.amp.GradScaler()  # mixed precision


# ===========================
# 11. TRAINING LOOP
# ===========================
for epoch in range(config.epochs):

    model.train()
    running_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for step, batch in enumerate(pbar):

        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss / config.accum_steps

        scaler.scale(loss).backward()

        # update model every accum_steps
        if (step + 1) % config.accum_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

        running_loss += loss.item() * config.accum_steps

        if step % 20 == 0:
            wandb.log({
                "train/loss": loss.item() * config.accum_steps,
                "lr": scheduler.get_last_lr()[0]
            })
            pbar.set_description(f"Epoch {epoch+1} Loss {loss.item():.4f}")

    # end of epoch logs
    wandb.log({"epoch_train_loss": running_loss / len(train_loader)})


    # ===========================
    # 12. VALIDATION
    # ===========================
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            preds = outputs.logits.argmax(dim=1)

            correct += (preds == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    wandb.log({"val/accuracy": val_acc})


# ===========================
# 13. Save Model + W&B Artifact
# ===========================
model.save_pretrained("bertweet_finetuned")
tokenizer.save_pretrained("bertweet_finetuned")

artifact = wandb.Artifact("bertweet-finetuned", type="model")
artifact.add_dir("bertweet_finetuned")
wandb.log_artifact(artifact)

wandb.finish()

wandb: Currently logged in as: youssefbouhachem2002 (youssefbouhachem2002-esprit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to 

Map:   0%|          | 0/1440000 [00:00<?, ? examples/s]

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1440000 [00:00<?, ? examples/s]

Map:   0%|          | 0/160000 [00:00<?, ? examples/s]

...

Validation Accuracy: 0.8868


Epoch 2 Loss 0.0649: 100%|██████████| 22500/22500 [48:34<00:00,  7.72it/s]


Validation Accuracy: 0.8913


wandb: Adding directory to artifact (bertweet_finetuned)... Done. 2.7s


epoch_train_loss,█▁
lr,▁▁▂▃▃████▇▇▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁
train/loss,▆▆▅▅▅█▆▄▇▄▃▄▃▆▆▂▄▃▃▅▃▃▃▄▄▃▂▆▁▄▃▃▃▄▄▃▂▃▃▄
val/accuracy,▁█
epoch_train_loss,0.24758
lr,0.0
train/loss,0.25963
val/accuracy,0.89131


**In this link you can see the model performance in WandB platform** :https://wandb.ai/youssefbouhachem2002-esprit/bertweet-sentiment140/runs/crdaamll?nw=nwuseryoussefbouhachem2002